In [48]:
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote
import folium, json

In [49]:
df = pd.read_csv('서울_주유소(230418).csv')
df.head()

,상호,주소,가격,셀프,상표,구
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),1669,Y,SK에너지,강남구
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),1684,Y,현대오일뱅크,강남구
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),1687,Y,GS칼텍스,강남구
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1704,Y,SK에너지,강남구


In [50]:
# 구별 집계 분석
gu_mm = df.groupby('구')['가격'].agg(['mean', 'min', 'max'])
gu_mm.head()

,mean,min,max
구,,,
강남구,1942.294118,1623,2578
강동구,1735.000000,1669,1978
강북구,1649.916667,1629,1707
강서구,1696.818182,1628,1995
관악구,1744.714286,1685,1852


In [51]:
# 해당구에서 최대값 최소값만 구하기
# 예 강남구
_, mi, ma = gu_mm.loc['강남구']    # _ is for mean, but we don't use it.
mi, ma = int(mi), int(ma)
mi, ma

(1623, 2578)

In [52]:
# 컬럼을 하나 추가하여 최대값인지 최소값인지 결정해서 넣을 예정
df['mm'] = np.zeros(df.shape[0], int)
df.head()

,상호,주소,가격,셀프,상표,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,0
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),1669,Y,SK에너지,강남구,0
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),1684,Y,현대오일뱅크,강남구,0
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),1687,Y,GS칼텍스,강남구,0
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1704,Y,SK에너지,강남구,0


In [53]:
# mm 컬럼에 최소값이 같으면 1을 넣고 최대값과 같으면 2를 넣을것
for gu in gu_mm.index:
    _, mi, ma = gu_mm.loc[gu]
    mi, ma = int(mi), int(ma)
    for i in df.index:
        if df.가격[i] == mi and df.구[i] == gu:
            df.mm[i] = 1
        if df.가격[i] == ma and df.구[i] == gu:
            df.mm[i] = 2

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6520\2950929148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 1
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6520\2950929148.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 2


In [54]:
df

,상호,주소,가격,셀프,상표,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),1669,Y,SK에너지,강남구,0
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),1684,Y,현대오일뱅크,강남구,0
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),1687,Y,GS칼텍스,강남구,0
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1704,Y,SK에너지,강남구,0
...,...,...,...,...,...,...,...
436,신내주유소,서울 중랑구 용마산로 705 (신내동),1668,Y,SK에너지,중랑구,0
437,(주)태영 구도일주유소 한인,서울 중랑구 망우로 170 (상봉동),1675,Y,S-OIL,중랑구,0
438,용마로주유소,서울 중랑구 용마산로 309 (면목동),1678,Y,SK에너지,중랑구,0
439,신일셀프주유소,서울 중랑구 상봉로 58 (망우동),1699,Y,SK에너지,중랑구,0


In [55]:
# mm컬럼에 데이터가 있는 행만 추출
df1 = df[df.mm !=0]
df1

,상호,주소,가격,셀프,상표,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),2578,N,GS칼텍스,강남구,2
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1669,Y,현대오일뱅크,강동구,1
46,광성주유소,서울 강동구 올림픽로 673 (천호동),1978,N,S-OIL,강동구,2
47,SK에너지(주) 매일주유소,서울 강북구 삼양로 217 (미아동),1629,Y,SK에너지,강북구,1
...,...,...,...,...,...,...,...
417,(주)대양씨앤씨 사직주유소,서울 종로구 사직로 65 (사직동),2240,N,GS칼텍스,종로구,2
418,다산주유소,서울 중구 다산로 233,1649,Y,S-OIL,중구,1
427,서남주유소,서울 중구 통일로 30,2753,N,SK에너지,중구,2
428,(주)자연에너지 대창주유소,서울 중랑구 동일로 636 (면목동),1635,Y,현대오일뱅크,중랑구,1


In [56]:
df1.shape

(70, 7)

# 지도 시각화

In [57]:
with open('./kakaoAPI.txt') as file:
    kakao_key = file.read()

In [67]:
lat_list, lng_list = [], []
for i in df1.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df1.주소[i])}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df1.상호[i])

In [68]:
df1['위도'] = lat_list
df1['경도'] = lng_list

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6520\197933941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['위도'] = lat_list
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6520\197933941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['경도'] = lng_list


In [69]:
df1.head()

,상호,주소,가격,셀프,상표,구,mm,위도,경도
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1623,Y,SK에너지,강남구,1,37.467272,127.118743
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),2578,N,GS칼텍스,강남구,2,37.511551,127.047115
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),1669,Y,현대오일뱅크,강동구,1,37.536417,127.149372
46,광성주유소,서울 강동구 올림픽로 673 (천호동),1978,N,S-OIL,강동구,2,37.541577,127.124242
47,SK에너지(주) 매일주유소,서울 강북구 삼양로 217 (미아동),1629,Y,SK에너지,강북구,1,37.622765,127.019086


In [70]:
filename = './skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [77]:
map = folium.Map(location = [37.5502, 126.982], zoom_start=12)
#df1.위도[i], df1.경도[i]
loc_lis = []
for i in df1.index:
    folium.Marker(
                 location=[df1.위도[i], df1.경도[i]],
                           popup = folium.Popup(df1.주소[i], max_width=200),
#                            tooltip = f'{df1.상호[i]}<br>휘발유:{df1.가격[i]:, d}원',
                           icon = folium.Icon(color='green' if df.mm[i] == 1 else 'red', 
                                             icon = 'glyphicon glyphicon-thumbs-up'
                                             if df.mm[i] == 1 else 'glyphicon glyphicon-thumbs-up')
                          ).add_to(map)

# create title
title = '주유소가격'
map.get_root().html.add_child(folium.Element(title))
map

In [78]:
filename = './skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [79]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=12)
for i in df1.index:
    folium.Marker(
        location=[df1.위도[i], df1.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df1.상호[i]}<br>휘발유:{df1.가격[i]:,d}원',
        icon=folium.Icon(color='green' if df1.mm[i] == 1 else 'red',
                        icon = 'glyphicon glyphicon-thumbs-up' 
            if df1.mm[i] == 1 else 'glyphicon glyphicon-thumbs-down')
    ).add_to(map)
map